## Longest isoform NCBI from gff

In [1]:
from Bio import SeqIO
import os
import gffutils

In [ ]:
# This script takes a while (aprox 30 mins for the 20 sp). For chick, the number of isoforms output is the same as the number of rows in the OMA
# splice file.
#
# TO DO:
# - Implement the option of selecting specific folder in which to run the script (e.g. maintain 
# the option of going through every GCF folder, but also be able to choose which GCF folder to 
# run the script only)


In [9]:
## for only one GCF_0... folder ## ciona in this case (was missing from previous one)
 
gene_to_prot= {}
prot_to_gene = {}
folder_path = '/directoryto/ncbi_dataset/data/GCFs/GCF_000224145.1'
if os.path.isdir(folder_path):
    gff = folder_path+'/genomic.gff'
    file = folder_path+'/protein.faa'
    db = gffutils.create_db(gff, ':memory:', merge_strategy="create_unique", keep_order=True)
        # Loop through all genes
    for t in db.features_of_type('gene', order_by='start'):
        gene = t.id
        gene_list = []
        ordered_child = list(db.children(t, featuretype='CDS', order_by='start'))
            
        # Loop through all children of genes
        for child in ordered_child:
            type_attribute = ['protein_id', 'Name']
                
            # Loop through all proteins of children??
            for att_type in type_attribute:
                protein = child.attributes.get(att_type, [None])[0]
                if protein:
                    break
                        
            if not protein:
                print('warning')
                print(child)
                continue
                
            corr_gene = prot_to_gene.get(protein, None)
            #if there was already a gene for this protein, replace it for the previous one
            if corr_gene and corr_gene!=gene:
                gene = corr_gene
                # now make sure the other proteins in the same gene (gene_list-a list of the proteins inside this gene)
                # have the same gene associated
                for other_prot in gene_list: 
                    prot_to_gene[other_prot] = gene
                gene_list = gene_to_prot[gene]+gene_list #Add this protein and the corresponding previous ones to the list of prots in this gene
            else:
                prot_to_gene[protein] = gene
            if protein not in gene_list:
                gene_list.append(protein)
        if len(gene_list)!=0:
            gene_to_prot[gene] = gene_list
    #print(prot_to_gene)    
    
    # Now we have the gene_to_prot and prot_to_gene information for this current folder (species), let's proceed to select the
    # longest one using the .faa file.
seen = set()
mydict = {}
for seq_record in SeqIO.parse(file, 'fasta'):
    sequence_len = len(seq_record.seq)
    gene = prot_to_gene[seq_record.id]
    
    if gene not in seen:
        mydict[gene]= {'length': sequence_len, 'ID': seq_record.id}
        seen.add(gene)
    elif mydict[gene]['length'] < sequence_len:
        mydict.pop(gene)
        mydict[gene]= {'length': sequence_len, 'ID': seq_record.id}
            
#Now that we have a dictionary with each gene and its longest isoform, let's create the output fasta with them
accession_numbers = []
sequences=[]
for key in mydict:
    accession_numbers.append(mydict[key]['ID'])
        
for seq_record in SeqIO.parse(file, 'fasta'):
    if seq_record.id in accession_numbers:
        sequences.append(seq_record)
            
with open(folder_path+'/longestiso.faa', 'w') as outfile:
    SeqIO.write(sequences, outfile, "fasta")

In [8]:
##### SPECIFIC FOR NCBI. All folders in original_dir need to be genomes folders with genomic.gff (& protein.faa) 
##### inside                                                                                            


original_dir = '/directoryto/ncbi_dataset/data/GCFs/'

for folder in os.listdir(original_dir):
    gene_to_prot= {}
    prot_to_gene = {}
    folder_path = original_dir + folder
    if os.path.isdir(folder_path):
        gff = folder_path+'/genomic.gff'
        file = folder_path+'/protein.faa'
        db = gffutils.create_db(gff, ':memory:', merge_strategy="create_unique", keep_order=True)
        # Loop through all genes
        for t in db.features_of_type('gene', order_by='start'):
            gene = t.id
            gene_list = []
            ordered_child = list(db.children(t, featuretype='CDS', order_by='start'))
            
            # Loop through all children of genes
            for child in ordered_child:
                type_attribute = ['protein_id', 'Name']
                
                # Loop through all proteins of children??
                for att_type in type_attribute:
                    protein = child.attributes.get(att_type, [None])[0]
                    if protein:
                        break
                        
                if not protein:
                    print('warning')
                    print(child)
                    continue
                
                corr_gene = prot_to_gene.get(protein, None)
                #if there was already a gene for this protein, replace it for the previous one
                if corr_gene and corr_gene!=gene:
                    gene = corr_gene
                    # now make sure the other proteins in the same gene (gene_list-a list of the proteins inside this gene)
                    # have the same gene associated
                    for other_prot in gene_list: 
                        prot_to_gene[other_prot] = gene
                    gene_list = gene_to_prot[gene]+gene_list #Add this protein and the corresponding previous ones to the list of prots in this gene
                else:
                    prot_to_gene[protein] = gene
                if protein not in gene_list:
                    gene_list.append(protein)
            if len(gene_list)!=0:
                gene_to_prot[gene] = gene_list
    #print(prot_to_gene)    
    
    # Now we have the gene_to_prot and prot_to_gene information for this current folder (species),
    # let's proceed to select the longest one using the .faa file.
    seen = set()
    mydict = {}
    for seq_record in SeqIO.parse(file, 'fasta'):
        sequence_len = len(seq_record.seq)
        gene = prot_to_gene[seq_record.id]
    
        if gene not in seen:
            mydict[gene]= {'length': sequence_len, 'ID': seq_record.id}
            seen.add(gene)
        elif mydict[gene]['length'] < sequence_len:
            mydict.pop(gene)
            mydict[gene]= {'length': sequence_len, 'ID': seq_record.id}
            
    #Now that we have a dictionary with each gene and its longest isoform, 
    # let's create the output fasta with them
    accession_numbers = []
    sequences=[]
    for key in mydict:
        accession_numbers.append(mydict[key]['ID'])
        
    for seq_record in SeqIO.parse(file, 'fasta'):
        if seq_record.id in accession_numbers:
            sequences.append(seq_record)
            
    with open(folder_path+'/longestiso.faa', 'w') as outfile:
        SeqIO.write(sequences, outfile, "fasta")
        

NameError: name 'file' is not defined

In [2]:
import pandas as pd
corr=pd.read_csv('/directoryto/ncbi_dataset/data/SpeciesInfo.csv')

In [3]:
corr.rename(columns={"Correct NCBI refseq": "Correct_NCBI_refseq"}, inplace=True)

In [4]:
NCBIrs= list(corr["Correct_NCBI_refseq"])
NCBIrs = [item for item in NCBIrs if not(pd.isnull(item)) == True]
spcode=list(corr["Code"])
spcode=[item for item in spcode if not(pd.isnull(item)) == True]

In [10]:
import shutil
from os.path import exists

for (ref, code) in zip(NCBIrs,spcode):
    original = r'/directoryto/ncbi_dataset/data/GCFs/' + ref +'/longestiso.faa'
    target = r'/directoryto/orthofinder_runs/OrthoFinder/topNCBI20/longestisos/'+ code + '.fa'
    isExisting = os.path.exists(original)
    if not isExisting:
        print('This file does not exist')
        print(original)
        continue
    shutil.copyfile(original, target)